# Utils

In [ ]:
from copy import deepcopy
import os
from typing import Dict, List

import numpy as np
import pandas as pd
import scipy as sp
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC
import tsfresh
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, EfficientFCParameters, MinimalFCParameters
from tsfresh.feature_selection.relevance import calculate_relevance_table
from tsfresh.transformers import RelevantFeatureAugmenter, FeatureAugmenter, FeatureSelector

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [ ]:
# parameters for saving data
PROCESSED_DATA_DIR = "processed_data"

# Automated feature extraction

## Utilities and preprocessing

In [ ]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # make sure that data has minute resolution with no missing parts from
    # start to end, with no missing values
    df = fill_missing_activity(df)
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def get_clean_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, List[pd.DataFrame]]:
    """
    Cleans DataFrames, filling missing values and resampling with given
    frequency.
    
    Returns three lists of DataFrames:
    - full 24hs
    - days: [8:00, 21:00)
    - nights: [21:00, 8:00)
    
    :param dfs: list of DataFrames to clean; each one has to have "timestamp"
    and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "full_24h", "day" and "night", corresponding
    to data from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]
    
    night_dfs = [get_day_part(df, part="night") for df in full_dfs]
    day_dfs = [get_day_part(df, part="day") for df in full_dfs]

    datasets = {
        "full_24h": full_dfs,
        "night": night_dfs,
        "day": day_dfs
    }

    return datasets


def get_tsfresh_flat_format_df(dfs: List[pd.DataFrame]) -> pd.DataFrame:
    """
    Creates DataFrame in a "flat" format for tsfresh from list of DataFrames.
    Each one is assumed to have "timestamp" and "activity" columns.
    
    :param dfs: list of DataFrames; each one has to have "timestamp" and
    "activity" columns
    :returns: DataFrame in tsfresh "flat" format
    """
    dfs = deepcopy(dfs)  # create copy to avoid side effects
    
    flat_df = pd.DataFrame(columns=["id", "timestamp", "activity"])

    for idx, df in enumerate(dfs):
        df["id"] = idx
        flat_df = flat_df.append(df)

    flat_df = flat_df.reset_index(drop=True)
        
    return flat_df

## Parameters and constants

In [ ]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=5000
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=500,
        criterion="entropy"
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": [None, "balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "C": np.logspace(10e-3, 10e3, num=50),
        "gamma": np.logspace(10e-3, 10e3, num=50),
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    }
}

## tsfresh

### Utilities

In [ ]:
def extract_tsfresh_features(dfs: List[pd.DataFrame], settings: Dict) \
        -> pd.DataFrame:
    """
    Performs feature extraction (only extraction, not selection) using tsfresh.
    
    :param dfs: list of DataFrames with time series, each with "timestamp" and
    "activity" columns
    :param settings: tsfresh settings, one of: ComprehensiveFCParameters,
    EfficientFCParameters, MinimalFCParameters
    :returns: DataFrame with extracted features, with one row per original
    DataFrame with time series (in the same order)
    """
    ts = get_tsfresh_flat_format_df(dfs)
    ids = ts["id"].unique()
    X = pd.DataFrame(index=ids)
    
    augmenter = FeatureAugmenter(
        default_fc_parameters=settings,
        column_id="id",
        column_sort="timestamp",
        column_value="activity",
        chunksize=1,
        n_jobs=4
    )
    
    augmenter.set_timeseries_container(ts)
    X = augmenter.transform(X)
    
    return X


class IncreasingFDRFeatureSelector(BaseEstimator, TransformerMixin):
    """
    Selects features using tsfresh feature selector and increasing FDR, if no
    features are selected at default FDR=0.05.
    """
    def __init__(self, verbose: bool = False):
        self.selector: FeatureSelector = None
        self.verbose: bool = verbose

    def fit(self, X, y):
        final_alpha = None
        for alpha in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                      0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]:
            self.selector = FeatureSelector(
                fdr_level=alpha,
                n_jobs=4,
                chunksize=1
            )
            self.selector.fit(X, y)
            if len(self.selector.relevant_features) > 0:
                if self.verbose:
                    print("FDR:", final_alpha)
                return selector

        raise ValueError("Failed to select any features")
    
    def transform(self, X):
        return self.selector.transform(X)


class TsfreshTopNFeatureSelector(BaseEstimator, TransformerMixin):
    """
    Selects top N features using tsfresh feature selector.
    """
    def __init__(self, n: int = 10):
        self.n: int = n
        self.features: List[int] = None
    
    def fit(self, X, y):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        
        if not isinstance(y, pd.Series):
            y = pd.Series(y)
        
        relevance_table = calculate_relevance_table(X, y)
        relevance_table.sort_values("p_value", inplace=True)
        features = relevance_table.head(self.n)["feature"]
        self.features = list(features.values)
    
    def transform(self, X, y=None):
        return X[:, self.features]

### Feature extraction

In [ ]:
dataset_str = "psykose"  # "depresjon" or "psykose"

In [ ]:
dataset = Dataset(dirpath=os.path.join("data", dataset_str))
condition = dataset.condition
control = dataset.control

In [ ]:
condition_parts_dfs = get_clean_dataframes(condition, freq="min")
control_parts_dfs = get_clean_dataframes(control, freq="min")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_dfs_list = condition_parts_dfs[part]
    control_dfs_list = control_parts_dfs[part]
    
    dfs_list = condition_dfs_list + control_dfs_list
    datasets[part] = dfs_list

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, f"{dataset_str}_y.csv"), header=None, dtype=int)
y = y.values.ravel()

In [ ]:
settings_dict = {"minimal": MinimalFCParameters(),
                 "efficient": EfficientFCParameters()}

for part, dfs in datasets.items():
    for settings_name, settings in settings_dict.items():
        X = extract_tsfresh_features(dfs, settings)
        filename = f"automatic_{dataset_str}_{settings_name}_{part}.csv"
        filepath = os.path.join(PROCESSED_DATA_DIR, filename)
        X.to_csv(filepath, index=False)

### Minimal settings

In [ ]:
dataset_str = "depresjon"  # "depresjon" or "psykose"

In [ ]:
for part in ["full_24h", "night", "day"]:
    print(f"PART: {part}")
    
    filename = f"automatic_{dataset_str}_minimal_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    X = pd.read_csv(filepath, header=0).fillna(0).values
    
    y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, f"{dataset_str}_y.csv"), header=None, dtype=int)
    y = y.values.ravel()

    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

### Efficient settings, increasing FDR

In [ ]:
dataset_str = "depresjon"  # "depresjon" or "psykose"

In [ ]:
for part in ["full_24h", "night", "day"]:
    print(f"PART: {part}")
    
    filename = f"automatic_{dataset_str}_efficient_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    X = pd.read_csv(filepath, header=0).fillna(0).values
    
    y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, f"{dataset_str}_y.csv"), header=None, dtype=int)
    y = y.values.ravel()

    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            
            selector = IncreasingFDRFeatureSelector(verbose=True)
            selector.fit(X_train, y_train)
            X_train, X_test = selector.transform(X_train), selector.transform(X_test)
            
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

### Efficient settings, top N features

In [ ]:
dataset_str = "depresjon"  # "depresjon" or "psykose"

top_n = 5

In [ ]:
for part in ["full_24h", "night", "day"]:
    print(f"PART: {part}")
    
    filename = f"automatic_{dataset_str}_efficient_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    X = pd.read_csv(filepath, header=0).fillna(0).values
    
    y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, f"{dataset_str}_y.csv"), header=None, dtype=int)
    y = y.values.ravel()

    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)

            selector = TsfreshTopNFeatureSelector(n=top_n)
            selector.fit(X_train, y_train)
            X_train, X_test = selector.transform(X_train), selector.transform(X_test)
            
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()